In [1]:
#需要修改的内容
#列表title
title0 = '特力A(000025)股吧'
#爬取的股票代码
stkcd = '000025'
#总页数
page_num = 1785



import requests
import logging
import os
from tqdm import tqdm
from lxml import etree
from concurrent.futures import ThreadPoolExecutor

# 配置日志
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('crawler.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# 隧道域名:端口号
tunnel = "155991151.sd.proxy.xiequ.cn:2829"
# 用户名密码方式
username = "hughgenius"
password = "Li566626"
proxies = {
    "http": f"http://{username}:{password}@{tunnel}/",
    "https": f"http://{username}:{password}@{tunnel}/"
}

class GubaDownloader:
    def __init__(self, max_workers=64):
        if not os.path.exists("data/pages"):
            os.makedirs("data/pages")

        self.session = requests.Session()
        self.session.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'zh-CN,zh;q=0.9'
        }
        self.max_workers = max_workers
        self.failed_pages = set()
        self.max_retries_per_page = 10  # 每个页面的最大重试次数

    def check_page_valid(self, html_content):
        """检查页面是否有效"""
        try:
            tree = etree.HTML(html_content)
            title = tree.xpath('//title/text()')
            return bool(title and title0 in title[0])
        except:
            return False

    def download_page(self, page_num, pbar):
        """下载单个页面"""
        file_path = f"data/pages/{stkcd}_{page_num}.html"

        # 如果文件已存在且大小正常，跳过下载
        if os.path.exists(file_path) and os.path.getsize(file_path) > 1000:
            pbar.update(1)
            return True

        url = f"https://guba.eastmoney.com/list,{stkcd},f_{page_num}.html"
        if page_num == 1:
            url = f"https://guba.eastmoney.com/list,{stkcd},f_1.html"

        for i in range(self.max_retries_per_page):
            try:
                response = self.session.get(url, proxies = proxies, timeout=5)
                if response.status_code == 200:
                    html_content = response.text
                    if self.check_page_valid(html_content):
                        with open(file_path, 'w', encoding='utf-8') as f:
                            f.write(html_content)
                        pbar.update(1)
                        return True
            except Exception as e:
                logging.info(f"下载页面 {page_num} 失败: {e}")

        # 记录失败的页面
        self.failed_pages.add(page_num)
        return False

    def retry_failed_pages(self):
        """重试失败的页面"""
        if not self.failed_pages:
            return

        failed_pages = list(self.failed_pages)
        self.failed_pages.clear()

        print(f"\n开始重试失败页面，共 {len(failed_pages)} 页")
        with tqdm(total=len(failed_pages), desc="重试进度") as pbar:
            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                futures = []
                for page_num in failed_pages:
                    future = executor.submit(self.download_page, page_num, pbar)
                    futures.append(future)

                for future in futures:
                    future.result()

        # 如果还有失败的页面，再次重试
        if self.failed_pages:
            print(f"\n仍有 {len(self.failed_pages)} 页下载失败，进行最后一次重试")
            self.retry_failed_pages()

    def run(self, start_page=1, end_page=94481):
        """运行下载器"""
        total_pages = end_page - start_page + 1

        print(f"开始下载页面，范围：{start_page} - {end_page}")
        with tqdm(total=total_pages, desc="下载进度") as pbar:
            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                futures = []
                for page_num in range(start_page, end_page + 1):
                    future = executor.submit(self.download_page, page_num, pbar)
                    futures.append(future)

                for future in futures:
                    future.result()

        # 重试失败的页面
        if self.failed_pages:
            self.retry_failed_pages()

        print("\n下载完成")
        if self.failed_pages:
            print(f"仍有 {len(self.failed_pages)} 页下载失败：{sorted(list(self.failed_pages))}")

def main():

    try:
        downloader = GubaDownloader(max_workers = 32)
        downloader.run(start_page=1, end_page=page_num)
    except KeyboardInterrupt:
        print("\n程序被用户中断")
    except Exception as e:
        logging.error(f"程序异常: {e}")

if __name__ == "__main__":
    main()

开始下载页面，范围：1 - 1785


下载进度:  90%|█████████ | 1609/1785 [04:41<00:30,  5.71it/s]



开始重试失败页面，共 176 页


重试进度:  94%|█████████▍| 166/176 [00:45<00:02,  3.61it/s]



仍有 10 页下载失败，进行最后一次重试

开始重试失败页面，共 10 页


重试进度:  60%|██████    | 6/10 [00:18<00:12,  3.13s/it]



仍有 4 页下载失败，进行最后一次重试

开始重试失败页面，共 4 页


重试进度: 100%|██████████| 4/4 [00:28<00:00,  7.02s/it]


下载完成
